In [1]:
import pandas as pd
from ast import literal_eval
from random import choice

df_wiki = pd.read_csv("../dat/wiki/graph.csv")
df_llm = pd.read_json("../dat/llm/graph.json")
df_llm['deps'] = df_llm['dep_articles']
df_txb = pd.read_json("../dat/textbooks/graph.json").iloc[:,[0,3]]
df_txb.columns=['concept','deps']
df_wiki['deps'] = [[dep[1] for dep in literal_eval(deps)] for deps in (df_wiki.dep_articles)]

In [2]:
n=100
rnd_st=123
concepts = []
deps = []

for df in [df_llm, df_wiki,df_txb]:
    df = df.loc[df.deps.str.len() > 0]
    samp = df.sample(n,random_state=rnd_st)
    concepts.extend(samp['concept'])
    deps.extend([choice(i) for i in samp['deps']])
    
len(deps) == len(concepts)

True

In [3]:
samples = pd.DataFrame({"concept":concepts,"dependency":deps,"source":["llm"]*n+['wiki']*n+['textbook']*n})
samples = samples.sample(frac=1).reset_index(drop=True)
samples['source'].to_csv('../dat/validation/source_column.csv')
samples.loc[:,['concept','dependency']].to_excel('../dat/validation/deps_to_validate.xlsx')